# Tokyo Foursquare Dataset EDA  

Foursquare check-ins in Tokyo from around 2012 - 2013. Goal is to come up with a visualization for subway / metro travel in this time.

In [3]:
import pandas as pd
import numpy as np

In [103]:
import json

In [4]:
df_tokyo = pd.read_csv('../_data/foursquare-nyc-and-tokyo-check-ins/dataset_TSMC2014_TKY.csv')

In [5]:
df_trains = df_tokyo[df_tokyo['venueCategory'] == 'Train Station']

In [6]:
df_subway = df_tokyo[df_tokyo['venueCategory'] == 'Subway']

In [7]:
df_trains.head(2)

userId                   venueId           venueCategoryId  venueCategory  \
7      114  4b3eae5cf964a520b4a025e3  4bf58dd8d48988d129951735  Train Station   
15    2290  4b53b05ef964a520e8a727e3  4bf58dd8d48988d129951735  Train Station   

     latitude   longitude  timezoneOffset                    utcTimestamp  
7   35.700253  139.480255             540  Tue Apr 03 19:35:36 +0000 2012  
15  35.749538  139.586540             540  Tue Apr 03 20:14:18 +0000 2012

### Convert utc to regular time

In [8]:
timezone_offset = pd.to_timedelta('6 hours')

def convert_time(row):
    return pd.to_datetime(row['utcTimestamp']) + timezone_offset

In [9]:
df_trains['localTimestamp'] = df_trains.apply(convert_time, axis=1)

/Users/michael/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
df_subway['localTimestamp'] = df_subway.apply(convert_time, axis=1)

/Users/michael/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [69]:
df_trains[0:1].reset_index()

0   2012-04-04 01:35:36
Name: localTimestamp, dtype: datetime64[ns]

## Function to convert to geojson

In [11]:
def row_to_geojson_full(row, json_list=[]):
    headers = row.columns
    properties = {}
    for h in headers:
        properties[h] = row[h].values[0]
    dict = {"type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [row['longitude'].values[0], row['latitude'].values[0]]
            },
            "properties": properties}
    json_list.append(dict)

In [12]:
def row_to_geojson(row, json_list=[]):
#     print(row)
#     dict = {"type": "Feature",
#             "geometry": {
#                 "type": "Point",
#                 "coordinates": [row['longitude'].values[0], row['latitude'].values[0]]
#             },
#             "properties": {
#                 "localTimestamp": row['localTimestamp'].values[0]
#             }}
    dict = {"type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [row['longitude'], row['latitude']]
            },
            "properties": {
                "localTimestamp": row['localTimestamp']
            }}
    json_list.append(dict)

In [13]:
# json.dumps(dict, default=timestamp_to_json)
def timestamp_to_json(obj):
    if isinstance(obj, pd.datetime):
        return obj.__str__()

### Test Subway JSON collection

In [15]:
list_json_subways = []
df_subway[:10].apply(lambda x: row_to_geojson(x, json_list=list_json_subways), axis=1)

32     None
34     None
54     None
65     None
75     None
92     None
93     None
105    None
108    None
125    None
dtype: object

In [16]:
list_json_subways

[{'geometry': {'coordinates': [139.7195989, 35.74880451], 'type': 'Point'},
  'properties': {'localTimestamp': Timestamp('2012-04-04 02:59:52')},
  'type': 'Feature'},
 {'geometry': {'coordinates': [139.798767, 35.68220662], 'type': 'Point'},
  'properties': {'localTimestamp': Timestamp('2012-04-04 03:04:04')},
  'type': 'Feature'},
 {'geometry': {'coordinates': [139.633677, 35.55002896], 'type': 'Point'},
  'properties': {'localTimestamp': Timestamp('2012-04-04 03:39:12')},
  'type': 'Feature'},
 {'geometry': {'coordinates': [139.7113779, 35.73027306], 'type': 'Point'},
  'properties': {'localTimestamp': Timestamp('2012-04-04 03:52:17')},
  'type': 'Feature'},
 {'geometry': {'coordinates': [139.7106913, 35.68859747], 'type': 'Point'},
  'properties': {'localTimestamp': Timestamp('2012-04-04 03:56:43')},
  'type': 'Feature'},
 {'geometry': {'coordinates': [139.8175974, 35.66959007], 'type': 'Point'},
  'properties': {'localTimestamp': Timestamp('2012-04-04 04:06:34')},
  'type': 'Featu

## Reduce coordinate noise

Basically try to map check-ins to a reasonable number. Check-ins might be referring to the same station, but slightly off in GPS coordinates, so experiment with how precise the geocoordinates should be. Assuming there are about 140 stations to work with.

In [17]:
len(df_subway['latitude'].apply(lambda x: np.round_(x, decimals=3)).value_counts())

136

In [18]:
len(df_subway['longitude'].apply(lambda x: np.round_(x, decimals=3)).value_counts())

164

In [23]:
df_subway_small = df_subway[['venueId', 'latitude', 'longitude', 'localTimestamp']]
df_subway_small['latitude'] = df_subway_small['latitude'].apply(lambda x: np.round_(x, decimals=3))
df_subway_small['longitude'] = df_subway_small['longitude'].apply(lambda x: np.round_(x, decimals=2))
len((df_subway_small['latitude'].map(str) + ":" + df_subway_small['longitude'].map(str)).unique())

/Users/michael/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/michael/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


292

In [290]:
df_subway_small['localTimestamp'].diff().std()

Timedelta('0 days 03:17:35.704839')

### Save CSV

In [25]:
df_subway_small.to_csv('../_data/df_subway_station_checkins')

## Extra stuff (ignorable):

### Find average time difference

Want to calculate the average time difference between check-ins, accounting for outliers

In [298]:
df_timedeltas_subway = df_subway_small['localTimestamp'].diff()[1:]

In [305]:
df_timedeltas_subway.std()

Timedelta('0 days 03:17:35.704839')

In [319]:
df_timedeltas_subway[df_timedeltas_subway <= df_timedeltas_subway.std()].mean()

Timedelta('0 days 00:04:39.152909')

In [274]:
len(df_subway_small['venueId'].unique())

431

In [258]:
row_to_geojson_full(df_subway_small[0:1])

{'geometry': {'coordinates': [139.72, 35.75], 'type': 'Point'},
 'properties': {'latitude': 35.75,
  'localTimestamp': numpy.datetime64('2012-04-04T02:59:52.000000000'),
  'longitude': 139.72},
 'type': 'Feature'}

In [321]:
json_list = []
df_subway_small[:100].apply(lambda x: row_to_geojson(x, json_list=json_list), axis=1)

32     None
34     None
54     None
65     None
75     None
92     None
93     None
105    None
108    None
125    None
126    None
130    None
138    None
155    None
156    None
166    None
181    None
190    None
191    None
192    None
217    None
224    None
235    None
236    None
237    None
239    None
240    None
242    None
251    None
254    None
       ... 
489    None
493    None
494    None
498    None
500    None
505    None
517    None
532    None
536    None
538    None
539    None
552    None
555    None
560    None
566    None
578    None
580    None
599    None
603    None
607    None
608    None
619    None
642    None
643    None
645    None
648    None
659    None
663    None
665    None
669    None
Length: 100, dtype: object

In [322]:
json.dumps(json_list, default=timestamp_to_json)

'[{"type": "Feature", "geometry": {"type": "Point", "coordinates": [139.72, 35.75]}, "properties": {"localTimestamp": "2012-04-04 02:59:52"}}, {"type": "Feature", "geometry": {"type": "Point", "coordinates": [139.8, 35.68]}, "properties": {"localTimestamp": "2012-04-04 03:04:04"}}, {"type": "Feature", "geometry": {"type": "Point", "coordinates": [139.63, 35.55]}, "properties": {"localTimestamp": "2012-04-04 03:39:12"}}, {"type": "Feature", "geometry": {"type": "Point", "coordinates": [139.71, 35.73]}, "properties": {"localTimestamp": "2012-04-04 03:52:17"}}, {"type": "Feature", "geometry": {"type": "Point", "coordinates": [139.71, 35.69]}, "properties": {"localTimestamp": "2012-04-04 03:56:43"}}, {"type": "Feature", "geometry": {"type": "Point", "coordinates": [139.82, 35.67]}, "properties": {"localTimestamp": "2012-04-04 04:06:34"}}, {"type": "Feature", "geometry": {"type": "Point", "coordinates": [139.86, 35.66]}, "properties": {"localTimestamp": "2012-04-04 04:06:57"}}, {"type": "Fe

In [318]:
df_subway_small[10:1000]['localTimestamp'].diff().std()

Timedelta('0 days 01:02:11.222440')

In [24]:
df_subway_small

venueId  latitude  longitude      localTimestamp
32      4b5da9e0f964a520bb6529e3    35.749     139.72 2012-04-04 02:59:52
34      4b8c5418f964a520e3ca32e3    35.682     139.80 2012-04-04 03:04:04
54      4b5bac4af964a520200f29e3    35.550     139.63 2012-04-04 03:39:12
65      4b7481e1f964a520bde02de3    35.730     139.71 2012-04-04 03:52:17
75      4b2692f9f964a520be7d24e3    35.689     139.71 2012-04-04 03:56:43
92      4b5599caf964a5209be827e3    35.670     139.82 2012-04-04 04:06:34
93      4bd6b3024e32d13a50f6c280    35.665     139.86 2012-04-04 04:06:57
105     4b9b8cd2f964a520c30e36e3    35.676     139.76 2012-04-04 04:12:01
108     4b66c862f964a520aa2a2be3    35.605     139.71 2012-04-04 04:13:27
125     4b5e34f8f964a520bb8329e3    35.696     139.75 2012-04-04 04:21:13
126     4b971b9ff964a5205ff834e3    35.685     139.77 2012-04-04 04:21:30
130     4dc5e8f5e4cd169dc659fe4d    35.792     139.65 2012-04-04 04:23:00
138     4b567b47f964a520651228e3    35.726     139.73 2012-04-04 04:25:09
155     4b669d8af964a52097252be3    35.647     139.75 2012-04-04 04:29:34
156     4b54fad0f964a52057d627e3    35.664     139.87 2012-04-04 04:29:46
166     4b467c9af964a5206c2226e3    35.682     139.80 2012-04-04 04:32:03
181     4b84c06cf964a520334331e3    35.701     139.76 2012-04-04 04:35:47
190     4b6a97d2f964a520a6d92be3    35.698     139.78 2012-04-04 04:37:25
191     4b6d4872f964a5202e6f2ce3    35.686     139.76 2012-04-04 04:37:30
192     4b54ff13f964a5201fd727e3    35.698     139.65 2012-04-04 04:37:35
217     4b78815ef964a520c9d22ee3    35.691     139.71 2012-04-04 04:44:43
224     4b24b5e7f964a5208d6924e3    35.726     139.72 2012-04-04 04:45:17
235     4b7cd447f964a520caa62fe3    35.673     139.72 2012-04-04 04:47:11
236     4b269340f964a520c27d24e3    35.678     139.71 2012-04-04 04:47:13
237     4b4487c9f964a5201ef625e3    35.677     139.74 2012-04-04 04:47:13
239     4b5cda0ef964a5205f4729e3    35.697     139.68 2012-04-04 04:47:19
240     4b669d8af964a52097252be3    35.647     139.75 2012-04-04 04:47:28
242     4b6fef86f964a5209a002de3    35.669     139.70 2012-04-04 04:49:17
251     4b18c616f964a520a7d523e3    35.672     139.79 2012-04-04 04:51:06
254     4dc5e8f5e4cd169dc659fe4d    35.792     139.65 2012-04-04 04:51:43
...                          ...       ...        ...                 ...
573265  4b55aa95f964a52007eb27e3    35.680     139.71 2013-02-14 16:51:57
573276  4b971b9ff964a5205ff834e3    35.685     139.77 2013-02-14 16:58:12
573282  4b58df1ff964a520137028e3    35.704     139.73 2013-02-14 16:59:42
573289  4ce4f953b43737046644f6ce    35.685     139.77 2013-02-14 17:03:09
573292  4b598750f964a520448b28e3    35.724     139.75 2013-02-14 17:03:46
573324  4b56b1f3f964a520d01828e3    35.689     139.83 2013-02-14 17:10:49
573340  4b4d9126f964a5202ed426e3    35.670     139.75 2013-02-14 17:14:26
573344  4b47b936f964a520f53b26e3    35.671     139.72 2013-02-14 17:15:20
573371  4b340afff964a5204b2425e3    35.682     139.79 2013-02-14 17:22:03
573393  4b70eccff964a5207e342de3    35.687     139.77 2013-02-14 17:30:17
573395  4b661945f964a5208e132be3    35.712     139.78 2013-02-14 17:31:57
573398  4b575064f964a520ce3028e3    35.692     139.78 2013-02-14 17:32:27
573401  50068d9ae4b042636d7fe03e    35.692     139.78 2013-02-14 17:33:00
573409  4b9b8cd2f964a520c30e36e3    35.676     139.76 2013-02-14 17:35:17
573476  4b55b89bf964a52026ed27e3    35.758     139.63 2013-02-14 17:52:33
573496  4b9b8cd2f964a520c30e36e3    35.676     139.76 2013-02-14 17:59:46
573502  4b7ba460f964a520a4692fe3    35.655     139.74 2013-02-14 18:02:29
573516  4b5a6863f964a52031c328e3    35.787     139.67 2013-02-14 18:05:48
573521  4e804d1c9a52c3af7b197bce    35.696     139.75 2013-02-14 18:06:28
573522  4b46d0ebf964a520982826e3    35.696     139.75 2013-02-14 18:06:33
573596  4b5d54a2f964a5200e5a29e3    35.701     139.74 2013-02-16 07:43:07
573598  4b5b04cff964a52026df28e3    35.702     139.75 2013-02-16 07:43:25
573603  4b5599caf964a5209be82